# The Rise and Fall of the Third Reich
# A History of Nazi Germany 
# by William L. Shirer
## A Textual Analysis

Read in file.

In [303]:
import pandas as pd
import book_reader as br
import book_viz as bv
import numpy as np
import json
import os
from IPython.display import display
rafo3r_text = open('rafo3r.txt', 'r', encoding='utf-8').read()
generate_book_df = False
generate_toc_df = False
generate_pivots = False
rafo3r_reader = br.book_reader('rafo3r', generate_book_df, generate_toc_df, generate_pivots)
book_file = 'rafo3r.txt'
h5_file = 'rafo3r.h5'
rafo3r, toc, book_pivot1, book_pivot2 = rafo3r_reader.main(book_file, h5_file)

Starting
Book processed
TOC processed
Chapter Markers made
Pivots processed


In [87]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('yg2bsm', '8e3m3cer5e')

In [93]:
display(rafo3r.head())

,Word,Stop Word,Count,Running Count,Position
0,william,False,12,1,0
1,l,True,0,0,1
2,shirer,False,5,1,2
3,the,True,0,0,3
4,rise,False,48,1,4


In [ ]:
rafo3r_sliced = pd.DataFrame()
for word in set(list(rafo3r[rafo3r['Stop Word'] == False]['Word'])): #iterate over unique words
    for i, v in enumerate(broken_list):   
    #if i % break_point == 0 or x == len(rafo3r):
        if i == 0:
            v0 = 0
        else:
            v0 = broken_list[i-1]
        rafo3r_sliced.loc[str(i),word] = rafo3r[rafo3r['Word'] == word][v0:v]['Running Count'].sum
display (rafo3r_sliced.head())

In [156]:
print(rafo3r[rafo3r['Word'] == 'germany'][1016 <= rafo3r['Position']][1016 <= rafo3r['Position']])
#print([1000:1200]['Running Count'].max())

In [177]:
def _count_within_range(word, v0, v):
    if len(rafo3r[rafo3r['Word'] == word][v0 < rafo3r['Position']][v >= rafo3r['Position']]['Running Count']) == 0:
        return 0
    return rafo3r[rafo3r['Word'] == word][v0 < rafo3r['Position']][v >= rafo3r['Position']]['Running Count'].max() - rafo3r[rafo3r['Word'] == word][v0 < rafo3r['Position']][v >= rafo3r['Position']]['Running Count'].min()

In [239]:
#places_json = json.loads('{"germany":["german"],"austria":[]}')
def word_vs_range_df_maker(book_df, word_json, break_point = 10000, min_count_req = 400):
    peak = len(book_df)
    broken_list = list(range(0,peak,break_point))
    broken_list.pop(0)#remove 0
    if broken_list[-1] != peak: broken_list.append(peak)
    plotter_df = pd.DataFrame()
    for word_main in word_json:
        these_words = word_json[word_main]
        for i, v in enumerate(broken_list):
            if i == 0:
                v0 = 0
            else:
                v0 = broken_list[i-1]
            plotter_df.loc[str(v),word_main] = _count_within_range(word_main,v0,v)
            #print(plotter2_df.at[str(v),place_main])
            #rafo3r[rafo3r['Word'] == place_main][v0:v]['Running Count'].max() - rafo3r[rafo3r['Word'] == place_main][v0:v]['Running Count'].min()
            for word_sub in these_words:
                plotter_df.loc[str(v),word_main] += _count_within_range(word_sub,v0,v)
#    plotter_df = plotter_df.drop(plotter_df.sum(axis=0)
#                                 [plotter_df.sum(axis=0)<min_count_req].index,axis=1)
    return plotter_df

In [295]:
places = open(os.getcwd() + os.sep + 'places.json', 'r').read()
places_json = json.loads(places)
people = open(os.getcwd() + os.sep + 'people.json', 'r').read()
people_json = json.loads(people)

In [3]:
rafo3r_viz = bv.book_viz(rafo3r, toc, p1, p2, places_json, people_json)
rafo3r_viz.places_vs_chapters()

VIZ


In [7]:
#rafo3r_viz.plotter_df.iplot(fill=True, filename='cuflinks/filled-area')

In [307]:
#using pivot2, create a new dataframe with words (subset based on places from json) as columns and chapter (counts)
#as rows.
def word_vs_chapter_df_maker(book_pivot2, word_json, ch_list, min_count_req = 400):
    bp2 = book_pivot2.copy()  #to my suprise, without this i was modifying the actual df (didnt think would happen in func)
    
    #######                      
    #remove this once you fix bug in pivot maker
    #print('P1',bp2.head())
    bp2.index = bp2.index.droplevel(1)
    bp2 = bp2[~bp2.index.duplicated(keep='first')]
    #print('P2',bp2.head())
    #######                      
    
    plotter_df = pd.DataFrame()
    for word_main in word_json:
        #print (word_main)
        other_words = word_json[word_main]
        s = 'Word == "%s"'%(word_main)
        #print (book_pivot2.query(s))
        #print (list(bp2.query(s).index)[:5])
        master_df = bp2.query(s).reset_index().set_index('Chapter')
        #try:
        master_df = master_df.reindex(ch_list).fillna(0)
        #except:
        #    #reach this if word appears in all chapters
        #    print ("reindex fail on",word_main)
        #    print (master_df.index)
        #    pass
        master_df['Word'] = word_main
        master_df.sort_index(inplace=True)
        try:
            del master_df['Book']
        except:
            #remove this try once you fix the issue that lets you remove the stuff at start.
            #issue is that you remove book up there so you cant delete it here.
            pass
        for word_sub in other_words:
            s = 'Word == "%s"'%(word_sub)
            minor_df = bp2.query(s).reset_index().set_index('Chapter')
            minor_df = minor_df.reindex(ch_list).fillna(0)
            minor_df.sort_index(inplace=True)
            try:
                del minor_df['Book']
            except:
                #remove this try once you fix the issue that lets you remove the stuff at start.
                #issue is that you remove book up there so you cant delete it here.
                pass
            master_df['Count'] = master_df['Count'] + minor_df['Count']
        plotter_df = pd.concat([plotter_df,master_df])
    plotter_df = plotter_df.reset_index()
    plotter_df.set_index(['Chapter', 'Word'], inplace=True)
    #print("for done")
    #print (plotter_df.head())
    #debug_df = plotter_df.copy()
    #try: 
    plotter_df = plotter_df.unstack(level=1)
    #except:
    #    pass
    #    print ("unstack failed")
    #    #need to figure out why this errors for the person json. 
    plotter_df = plotter_df.drop(plotter_df.sum(axis=0)
                                 [plotter_df.sum(axis=0)<min_count_req].index,axis=1)
    #try:
    plotter_df.columns = plotter_df.columns.droplevel(0)
    #except:
    #    pass
    #    print ("level drop failed")
    #    #need to figure out why this errors for the person json.
    return plotter_df#, debug_df

In [311]:
num_chapters = max(book_pivot2.reset_index()['Chapter'])
ch_list = list(range(1,num_chapters+1))

In [309]:
places_vs_chapter_df = word_vs_chapter_df_maker(book_pivot2, places_json, ch_list, 400)
#print (places_vs_chapter_df.head())
people_vs_chapter_df = word_vs_chapter_df_maker(book_pivot2, people_json, ch_list, 400)
#print (people_vs_chapter_df.head())
places_vs_range_df = word_vs_range_df_maker(rafo3r, places_json, 10000, 400)
#print (places_vs_range_df.head())
people_vs_range_df = word_vs_range_df_maker(rafo3r, people_json, 10000, 400)
#print (people_vs_range_df.head())

Word     austria  britain  czechoslovakia  france  germany  italy  poland  \
Chapter                                                                     
1            3.0      2.0             2.0     3.0     33.0    0.0     4.0   
2          104.0      2.0             9.0     2.0     76.0    0.0     0.0   
3           11.0      7.0             0.0     2.0    159.0    1.0     2.0   
4            5.0      3.0             0.0    15.0    166.0    0.0     4.0   
5           17.0     15.0             3.0    29.0    246.0    1.0     1.0   

Word     russia  united_states  
Chapter                         
1           5.0            2.0  
2           1.0            0.0  
3          11.0            6.0  
4          13.0            0.0  
5          13.0            2.0  
Word     goering  hitler  mussolini  ribbentrop
Chapter                                        
1            1.0    15.0        1.0         0.0
2            2.0   150.0        0.0         0.0
3            2.0   141.0        0.0  

/Users/lukestarnes/anaconda/envs/Py3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/lukestarnes/anaconda/envs/Py3.5/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



       austria  united_states  czechoslovakia  slovakia  hungary  britain  \
10000     43.0            1.0             3.0       0.0      0.0      1.0   
20000     60.0            0.0             3.0       0.0      0.0      5.0   
30000      5.0            4.0             0.0       0.0      0.0      0.0   
40000      0.0            0.0             0.0       0.0      0.0      0.0   
50000      7.0            0.0             0.0       0.0      0.0      3.0   

       russia  denmark  japan  belgium  sweeden  norway  poland  france  \
10000     3.0      0.0    0.0      0.0      0.0     0.0     3.0     2.0   
20000     2.0      0.0    0.0      0.0      0.0     0.0     0.0     0.0   
30000    10.0      0.0    0.0      0.0      0.0     0.0     1.0     0.0   
40000     4.0      0.0    0.0      0.0      0.0     0.0     3.0    13.0   
50000     8.0      0.0    0.0      0.0      0.0     0.0     0.0    11.0   

       germany  italy  
10000     69.0    0.0  
20000     92.0    0.0  
30000    114.0

In [115]:
#import cufflinks as cf
#init_notebook_mode()

In [116]:
#cf.getThemes()

['polar', 'white', 'solar', 'ggplot', 'pearl', 'space']

In [84]:
plotter2_df.iplot(kind='scatter', fill=True, filename='cufflinks/filled-area')

In [338]:
def _col_clean(name):
    return name.replace('_',' ').title()

['Austria', 'Britain', 'Czechoslovakia', 'France', 'Germany', 'Italy', 'Poland', 'Russia', 'United States']


In [340]:
c = 256 / len(places_vs_chapter_df.columns)
new_col_names = list(map(_col_clean, list(places_vs_chapter_df.columns)))

py.iplot([{
    'x': places_vs_chapter_df.index,
    'y': places_vs_chapter_df[col],
    'name': new_col_names[i],
    'fill' : 'tonexty',
    'line' : dict(color = ('rgb(%i, %i, 100)'%(int(c * i),int(255 - c * i)))),
}  for i, col in enumerate(places_vs_chapter_df.columns)], filename='plotly/places_vs_chapter')

In [230]:
#places_vs_chapter_df.iplot(kind='scatter', fill=True, filename='cufflinks/filled-area')

In [310]:
#people_vs_chapter_df.iplot(kind='scatter', fill=True, filename='cufflinks/filled-area')

c = 256 / len(people_vs_chapter_df.columns)
new_col_names = list(map(_col_clean, list(people_vs_chapter_df.columns)))

py.iplot([{
    'x': people_vs_chapter_df.index,
    'y': people_vs_chapter_df[col],
    'name': new_col_names[i],
    'fill' : 'tonexty',
    'line' : dict(color = ('rgb(%i, %i, 100)'%(int(c * i),int(255 - c * i)))),
}  for i, col in enumerate(places_vs_chapter_df.columns)], filename='plotly/places_vs_chapter')

In [341]:
#places_vs_range_df.iplot(kind='scatter', fill=True, filename='cufflinks/filled-area')

c = 256 / len(places_vs_range_df.columns)
new_col_names = list(map(_col_clean, list(places_vs_range_df.columns)))

py.iplot([{
    'x': places_vs_range_df.index,
    'y': places_vs_range_df[col],
    'name': new_col_names[i],
    'fill' : 'tonexty',
    'line' : dict(color = ('rgb(%i, %i, 100)'%(int(c * i),int(255 - c * i)))),
}  for i, col in enumerate(places_vs_range_df.columns)], filename='plotly/places_vs_range')

In [344]:
#people_vs_range_df.iplot(kind='scatter', fill=True, filename='cufflinks/filled-area')
c = 256 / len(people_vs_range_df.columns)
new_col_names = list(map(_col_clean, list(people_vs_range_df.columns)))

py.iplot([{
    'x': people_vs_range_df.index,
    'y': people_vs_range_df[col],
    'name': new_col_names[i],
    'fill' : 'tonexty',
    'line' : dict(color = ('rgb(%i, %i, 100)'%(int(c * i),int(255 - c * i)))),
}  for i, col in enumerate(people_vs_range_df.columns)], filename='plotly/people_vs_range')

       chamberlain  schulenburg  truman  jodl  hanfstaengl  rastenburg  \
10000          0.0          0.0     0.0   0.0          0.0         0.0   
20000          0.0          0.0     0.0   0.0          0.0         0.0   

       weizsaecker  papen  himmler  heydrich   ...    rundstedt  schleicher  \
10000          0.0    2.0      0.0       0.0   ...          0.0         1.0   
20000          0.0    0.0      0.0       0.0   ...          0.0         0.0   

       blomberg  beck  haider  schuschnigg  mussolini  kordt  molotov  rommel  
10000       0.0   0.0     0.0          0.0        0.0    0.0      0.0     0.0  
20000       0.0   0.0     0.0          0.0        0.0    0.0      0.0     0.0  

[2 rows x 48 columns]


In [ ]:
#table - top 5 people by chapter

In [ ]:
#%matplotlib inline
germany = rafo3r[rafo3r['Word'] == 'germany']
#c,bins = np.histogram(germany)
#germany.hist(bins=bins)



df.iplot(kind='scatter', filename='cufflinks/cf-simple-line')
data = [go.Scatter(x= germany.index,y=germany['Running Count'],name='Germany')]
#data2 = [go.Histogram(x= c,#germany['Running Occurance'],
#                      name='Germany')]
layout = go.Layout(title='scatter plot with pandas',
                   yaxis=dict(title='random distribution'), 
                   xaxis=dict(title='linspace'))
py.iplot(data, filename='pandas/basic-line-plot')#'basic-histogram'

In [90]:
#data = [go.Scatter(x= plotter_df.index,y=plotter_df['Count'],name='All')]
#data2 = [go.Histogram(x= c,#germany['Running Occurance'],
#                      name='Germany')]
layout = go.Layout(title='scatter plot with pandas',
                   yaxis=dict(title='random distribution'), 
                   xaxis=dict(title='linspace'))
#py.iplot(data, filename='pandas/basic-line-plot')#'basic-histogram'

In [312]:
grouped = rafo3r.groupby('Word')
grouped.describe()
#display(rafo3r.head())

Count       Position  Running Count
Word                                                 
           count   52.0      52.000000      52.000000
           mean     0.0  188255.538462       0.000000
           std      0.0  142844.055449       0.000000
           min      0.0     263.000000       0.000000
           25%      0.0   67038.500000       0.000000
           50%      0.0  170194.000000       0.000000
           75%      0.0  284169.500000       0.000000
           max      0.0  494003.000000       0.000000
$1         count    3.0       3.000000       3.000000
           mean     0.0  277315.333333       0.000000
           std      0.0  174176.980251       0.000000
           min      0.0  138672.000000       0.000000
           25%      0.0  179565.000000       0.000000
           50%      0.0  220458.000000       0.000000
           75%      0.0  346637.000000       0.000000
           max      0.0  472816.000000       0.000000
$10        count    1.0       1.000000       1.000000
           mean     0.0  475960.000000       0.000000
           std      NaN            NaN            NaN
           min      0.0  475960.000000       0.000000
           25%      0.0  475960.000000       0.000000
           50%      0.0  475960.000000       0.000000
           75%      0.0  475960.000000       0.000000
           max      0.0  475960.000000       0.000000
$100       count    1.0       1.000000       1.000000
           mean     0.0  507017.000000       0.000000
           std      NaN            NaN            NaN
           min      0.0  507017.000000       0.000000
           25%      0.0  507017.000000       0.000000
           50%      0.0  507017.000000       0.000000
...                 ...            ...            ...
‡ohlendorf std      NaN            NaN            NaN
           min      1.0  480630.000000       1.000000
           25%      1.0  480630.000000       1.000000
           50%      1.0  480630.000000       1.000000
           75%      1.0  480630.000000       1.000000
           max      1.0  480630.000000       1.000000
‡see       count    1.0       1.000000       1.000000
           mean     1.0  482917.000000       1.000000
           std      NaN            NaN            NaN
           min      1.0  482917.000000       1.000000
           25%      1.0  482917.000000       1.000000
           50%      1.0  482917.000000       1.000000
           75%      1.0  482917.000000       1.000000
           max      1.0  482917.000000       1.000000
‡the       count    2.0       2.000000       2.000000
           mean     2.0  370780.000000       1.500000
           std      0.0   97446.385515       0.707107
           min      2.0  301875.000000       1.000000
           25%      2.0  336327.500000       1.250000
           50%      2.0  370780.000000       1.500000
           75%      2.0  405232.500000       1.750000
           max      2.0  439685.000000       2.000000
‡this      count    1.0       1.000000       1.000000
           mean     1.0  222970.000000       1.000000
           std      NaN            NaN            NaN
           min      1.0  222970.000000       1.000000
           25%      1.0  222970.000000       1.000000
           50%      1.0  222970.000000       1.000000
           75%      1.0  222970.000000       1.000000
           max      1.0  222970.000000       1.000000

[180744 rows x 3 columns]

In [313]:
rafo3r['Count'].sort_values()[-10:] # top 10 values

568477    4222
136095    4222
399802    4222
502264    4222
321664    4222
103615    4222
253602    4222
11536     4222
51862     4222
27601     4222
Name: Count, dtype: int64

In [ ]:
rafo3r['Position'] = rafo3r.index

occurances = rafo3r.groupby('Word').size()
mega_words = occurances.index[occurances >= 1000]
rafo3r.index = rafo3r['Word']
#sw = rafo3r.ix['Stop Word']
#sw.head()

In [ ]:
def start_loc(x):
    return int(100 * min(x) / len(rafo3r))
def end_loc(x):
    return int(100 * max(x) / len(rafo3r))
rafo3r_pt = rafo3r[rafo3r['Stop Word'] == False].pivot_table(values='Position', 
            aggfunc=[len,start_loc,end_loc], index='Words')
rafo3r_pt.sort_values('len',ascending=False,inplace=True)
rafo3r_pt.rename(columns={'len': 'Count'}, inplace=True)
display(rafo3r_pt[:10])

rafo3r_pt2 = rafo3r[rafo3r['Stop Word'] == False].pivot_table(values='Position', 
             aggfunc=[len], index=['Word','Book','Chapter'])
rafo3r_pt2.sort_values('len',ascending=False,inplace=True)

display(rafo3r_pt2[:10])